In [1]:
import pandas as pd # type: ignore
import os # type: ignore
import opendatasets as od # type: ignore

In [2]:
dataset = 'https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset/data'

od.download(dataset)

Skipping, found downloaded files in ".\imdb-dataset" (use force=True to force download)


In [3]:
data_dir = './imdb-dataset'

In [4]:
os.listdir(data_dir)

['name.basics.tsv',
 'title.akas.tsv',
 'title.basics.tsv',
 'title.principals.tsv',
 'title.ratings.tsv']

In [5]:
roles_df = pd.read_table('./imdb-dataset/name.basics.tsv')

In [6]:
movie_df = pd.read_table('./imdb-dataset/title.basics.tsv')

C:\Users\adepa\AppData\Local\Temp\ipykernel_13936\2644435849.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_df = pd.read_table('./imdb-dataset/title.basics.tsv')


In [7]:
roles_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
...,...,...,...,...,...,...
13843801,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13843802,nm9993716,Essias Loberg,\N,\N,\N,\N
13843803,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
13843804,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


In [8]:
roles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13843806 entries, 0 to 13843805
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 633.7+ MB


In [9]:
roles_df = roles_df.drop(['nconst', 'knownForTitles'], axis = 1)

In [10]:
roles_df

,primaryName,birthYear,deathYear,primaryProfession
0,Fred Astaire,1899,1987,"actor,miscellaneous,producer"
1,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage"
2,Brigitte Bardot,1934,\N,"actress,music_department,producer"
3,John Belushi,1949,1982,"actor,writer,music_department"
4,Ingmar Bergman,1918,2007,"writer,director,actor"
...,...,...,...,...
13843801,Romeo del Rosario,\N,\N,"animation_department,art_department"
13843802,Essias Loberg,\N,\N,\N
13843803,Harikrishnan Rajan,\N,\N,cinematographer
13843804,Aayush Nair,\N,\N,cinematographer


In [11]:
roles_df = roles_df.replace(r'\N', 'None')

In [16]:
roles_df

,name_surname,birth,death,primary_profession,second_profession,third_profession
0,Fred Astaire,1899,1987,actor,miscellaneous,producer
1,Lauren Bacall,1924,2014,actress,soundtrack,archive_footage
2,Brigitte Bardot,1934,None,actress,music_department,producer
3,John Belushi,1949,1982,actor,writer,music_department
4,Ingmar Bergman,1918,2007,writer,director,actor
...,...,...,...,...,...,...
13843801,Romeo del Rosario,None,None,animation_department,art_department,None
13843802,Essias Loberg,None,None,None,None,None
13843803,Harikrishnan Rajan,None,None,cinematographer,None,None
13843804,Aayush Nair,None,None,cinematographer,None,None


In [12]:
roles_df[['primary_profession', 'second_profession', 'third_profession']] = roles_df['primaryProfession'].str.split(',', expand=True)

In [13]:
roles_df = roles_df.drop(['primaryProfession'], axis = 1)

In [14]:
roles_df = roles_df.rename(columns={'primaryName': 'name_surname',
                                    'birthYear': 'birth',
                                    'deathYear': 'death'})

In [17]:
for column in roles_df.columns:
    value_counts = roles_df[column].value_counts()
    if 'None' in value_counts:
        print(f"In column {column}, 'None' occurs {value_counts['None']} times.")
    else:
        print(f"In column {column}, 'None' not found.")

In column name_surname, 'None' occurs 50 times.
In column birth, 'None' occurs 13216739 times.
In column death, 'None' occurs 13609129 times.
In column primary_profession, 'None' occurs 2678482 times.
In column second_profession, 'None' not found.
In column third_profession, 'None' not found.
